# Rede Neural para Classificação Binária - Spam Detection Dataset

O dataset escolhido é o "Spam Detection dataset", aplicado à predição de mensagens spam com base em um conjunto de características, disponível no Kaggle (https://www.kaggle.com/datasets/smayanj/spam-detection-dataset). A variável alvo é `is_spam`, que assume os valores booleanos 0 (não spam) e 1 (spam). A análise foi realizada utilizando a linguagem de programação Python com as bibliotecas Pandas, Sklearn e Tensorflow.

1. Pré-processamento dos Dados

O dataset contém 20.000 amostras com as seguintes colunas:

*   num_links (int): número de links na mensagem
*   num_words (int): número total de palavras
*   has_offer (bool): presença de termos promocionais (ex: “oferta”)
*   sender_score (float): reputação do remetente
*   all_caps (bool): se o assunto da mensagem está em letras maiúsculas
*   is_spam (bool): variável alvo, determina se a mensagem é spam.
    *   A probabilidade da mensagem ser spam foi aumenta se:
        *   A quantidade de links for maior que 2
        *   Contém ‘oferta’ de algo
        *   Reputação do remetente for menor que 4
        *   Assunto estar com todas as letras em maiúsculo
        
Os fatores acima da variável `is_spam` foram combinados com pesos diferentes. Também foi adicionado um ruído usando aleatoriedade Gaussiana para simular incertezas do mundo real. Emails são marcados como spam se a probabilidade final passar de 0.5.

# Implementação
1. Instalação das bibliotecas

In [ ]:
pip install pandas scikit-learn tensorflow matplotlib seaborn

2. Funções de ativação

In [ ]:
import numpy as np

def sigmoid(x): return 1 / (1 + np.exp(-x))
def sigmoid_deriv(x): return sigmoid(x) * (1 - sigmoid(x))

def relu(x): return np.maximum(0, x)
def relu_deriv(x): return (x > 0).astype(float)

def tanh(x): return np.tanh(x)
def tanh_deriv(x): return 1 - np.tanh(x)**2

Funções sigmoid:

* sigmoid(x): Transforma valores em um intervalo entre 0 e 1. Boa para saída binária.
* sigmoid_deriv(x): derivada de x, é simples e usada na retropropagação.

Funções relu:
* relu(x): Retorna 0 para negativos e o valor original para positivos. Mais eficiente que sigmoid em camadas ocultas.
* relu_deriv(x): 1 para valores maiores que 0, senão 0.

Funções tanh:
* tanh(x): Saída entre -1 e 1. melhor centragem dos dados.
* tanh_deriv(x): derivada de x.


  3. Funções de perda

In [ ]:
import numpy as np

def mse_loss(y_true, y_pred): return np.mean((y_true - y_pred)**2)
def mse_loss_deriv(y_true, y_pred): return (y_pred - y_true)

def bce_loss(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-8, 1 - 1e-8)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

def bce_loss_deriv(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-8, 1 - 1e-8)
    return (y_pred - y_true) / (y_pred * (1 - y_pred) * len(y_true))


Funções MSE:
* apenas por completude e para fins comparativos

Funções BCE:
* Clipping (np.clip) é usado para evitar log(0) e erro numérico. Derivada ajustada para evitar divisão por zero e escalada com tamanho do batch (len(y_true)).

  4. Classe da rede neural

In [ ]:
import numpy as np
from activations import sigmoid, sigmoid_deriv, relu, relu_deriv, tanh, tanh_deriv
from losses import bce_loss, bce_loss_deriv, mse_loss, mse_loss_deriv

activation_functions = {
    'sigmoid': (sigmoid, sigmoid_deriv),
    'relu': (relu, relu_deriv),
    'tanh': (tanh, tanh_deriv)
}

loss_functions = {
    'bce': (bce_loss, bce_loss_deriv),
    'mse': (mse_loss, mse_loss_deriv)
}

class NeuralNetwork:
    def __init__(self, layers, activation, loss, lr):
        self.layers = layers
        self.lr = lr
        self.act, self.act_deriv = activation_functions[activation]
        self.loss, self.loss_deriv = loss_functions[loss]
        self.weights = []
        self.biases = []
        for i in range(len(layers) - 1):
            weight = np.random.randn(layers[i], layers[i+1]) * np.sqrt(2 / layers[i])
            bias = np.zeros((1, layers[i+1]))
            self.weights.append(weight)
            self.biases.append(bias)

    def forward(self, X):
        a = X
        self.zs, self.activations = [], [X]
        for i in range(len(self.weights)):
            z = np.dot(a, self.weights[i]) + self.biases[i]
            if i == len(self.weights) - 1:
                a = sigmoid(z)
            else:
                a = self.act(z)
            self.zs.append(z)
            self.activations.append(a)
        return a

    def backward(self, y):
        m = len(y)
        delta = self.activations[-1] - y
        deltas = [delta]
        for i in range(len(self.layers)-3, -1, -1):
            delta = np.dot(deltas[0], self.weights[i+1].T) * self.act_deriv(self.activations[i+1])
            deltas.insert(0, delta)
        for i in range(len(self.weights)):
            dw = np.dot(self.activations[i].T, deltas[i]) / m
            db = np.sum(deltas[i], axis=0, keepdims=True) / m
            self.weights[i] -= self.lr * dw
            self.biases[i] -= self.lr * db


    def train(self, X, y, epochs=100):
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = self.loss(y, y_pred)
            print(f"Epoch {epoch+1}, Loss: {loss:.6f}")
            self.backward(y)

    def predict(self, X):
        return (self.forward(X) > 0.5).astype(int)


A classe NeuralNetwork implementa uma rede neural multicamada para classificação binária, feita do zero com NumPy. O construtor `__init__` define a arquitetura com base na lista de camadas `layers`, função de ativação `activation`, função de perda `loss` e taxa de aprendizado `lr`. Os pesos são inicializados com distribuição normal escalada e os vieses como zeros.

O método forward executa a propagação direta, aplicando a função de ativação em cada camada, sendo sigmoid fixo na saída. O método backward realiza a retropropagação do erro, calculando os gradientes das funções de ativação e da função de perda para atualizar os pesos com gradiente descendente.

O método train executa o treinamento da rede por múltiplas épocas, imprimindo a perda a cada ciclo. Por fim, o método predict gera as previsões binárias com base na saída da rede (limiar de 0.5). A implementação permite testar diferentes ativações e perdas, mantendo o foco em problemas de classificação binária.

  5. Execução e treinamento do modelo

In [ ]:
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, f1_score
from neural_network import NeuralNetwork

valid_activations = ['sigmoid', 'tanh', 'relu']
valid_losses = ['bce', 'mse']

def main():
    if len(sys.argv) < 3:
        print("Uso: python main.py <activation> <loss>")
        print(f"Ativações válidas: {valid_activations}")
        print(f"Loss válidas: {valid_losses}")
        sys.exit(1)

    activation = sys.argv[1]
    loss = sys.argv[2]

    if activation not in valid_activations:
        print(f"Erro: ativação inválida '{activation}'. Use uma das {valid_activations}")
        sys.exit(1)

    if loss not in valid_losses:
        print(f"Erro: loss inválida '{loss}'. Use uma das {valid_losses}")
        sys.exit(1)

    df = pd.read_csv('spam.csv')
    X = df[['num_links', 'num_words', 'has_offer', 'sender_score', 'all_caps']].values
    y = df['is_spam'].astype(int).values.reshape(-1, 1)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.15, random_state=42
    )

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    nn = NeuralNetwork(layers=[5, 16, 8, 1], activation=activation, loss=loss, lr=0.5)
    nn.train(X_train, y_train, epochs=100)

    y_pred = nn.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print("\nMatriz de Confusão:")
    print(cm)
    print(f"F1 Score: {f1:.4f}")

if __name__ == "__main__":
    main()


Este código executa o treinamento e a avaliação de uma rede neural para classificação binária de spam, utilizando parâmetros passados pela linha de comando. Ele exige dois argumentos: a função de ativação (sigmoid, tanh ou relu) e a função de perda (bce ou mse). Os dados são carregados do arquivo spam.csv, normalizados com StandardScaler, e divididos em treino e teste. A rede neural é então criada com arquitetura [5, 16, 8, 1], treinada por 100 épocas com taxa de aprendizado 0.5. Após o treinamento, são exibidos a matriz de confusão e o F1 Score com base nas previsões feitas sobre os dados de teste. Isso permite testar diferentes configurações de rede diretamente via terminal.